## Jorge Pablo Ávila Gómez

# Ejercicio 1 (conteo de tweets): (1 punto)

Desarrollar  un  notebook  de  Jupyter,  denominado  “tweetCount.ipynb”  en  el  que  se  utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter, se establezca una duración de batch de un segundo, y se muestre, cada 5 segundos, el número de tweets recibidos en los últimos 10 segundos. ¿Alrededor de qué número (aproximado) se estabiliza el número de tweets que se procesan en el lapso de tiempo indicado (10 segundos)? ¿Tiene sentido? ¿Por qué?

In [1]:
import findspark

# Se indica la ruta de spark:
findspark.init("C:\\Users\\JorgeAvila\\Documents\\spark-2.4.7-bin-hadoop2.7")

import os

os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell"

In [2]:
# Se importan los diferentes paquetes necesarios:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

### Preparación del entorno Spark Streaming

In [3]:
sc = SparkContext("local[*]")
# Se inicializa SparkContext con la opción "local[*]" para que use todos los núcleos del equipo
ssc = StreamingContext(sc, batchDuration=1)
# Se crea el StreamingContext indicando que la duración del batch es 1s como pide el enunciado.
ssc.checkpoint("checkpoint")
# El uso de checkpoints es necesario cuando se van a realizar operaciones con ventanas.

### Conexión con kafka e introducción de los datos

In [4]:
# Para crear el stream de datos utilizamos la siguiente función:
tweetsDS = KafkaUtils.createDirectStream(
    ssc, # StreamingContext con el que conectar kafka
    ["kafkaTwitter"], # El topic con el que queremos conectar.
    {"bootstrap.servers": "192.168.1.100:9092, 192.168.1.100:9093"}, # Las direcciones de los brokers.
)

### Configuración de la ventana

Tenemos que configurar una ventana para poder calcular el número de tweets como nos indica el resultado. Necesitamos una ventana de tamaño 10 s, y que el desplazamiento de la ventana sea de 5 s.

In [5]:
# La ventana se crea con el siguiente código, indicando 10s de tamaño y un desplazamiento de 5s.
# Para cada ventana contamos el número de tweets que hemos encontrado y imprimimos en pantalla ese número.
tweetsDS.window(10, 5).count().pprint()

### Ejecución de la conexión

Antes de ejecutar la siguiente celda para que se inicie la conexión hay que preparar el entorno de kafka como se indica en la documentación del trabajo e iniciar el productor de tweets.

In [6]:
ssc.start()

-------------------------------------------
Time: 2021-01-09 13:44:24
-------------------------------------------
149

-------------------------------------------
Time: 2021-01-09 13:44:29
-------------------------------------------
237

-------------------------------------------
Time: 2021-01-09 13:44:34
-------------------------------------------
176

-------------------------------------------
Time: 2021-01-09 13:44:39
-------------------------------------------
184

-------------------------------------------
Time: 2021-01-09 13:44:44
-------------------------------------------
190

-------------------------------------------
Time: 2021-01-09 13:44:49
-------------------------------------------
195

-------------------------------------------
Time: 2021-01-09 13:44:54
-------------------------------------------
198

-------------------------------------------
Time: 2021-01-09 13:44:59
-------------------------------------------
187

-------------------------------------------
Time

In [7]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2021-01-09 13:46:09
-------------------------------------------
200

-------------------------------------------
Time: 2021-01-09 13:46:14
-------------------------------------------
201

-------------------------------------------
Time: 2021-01-09 13:46:19
-------------------------------------------
196

-------------------------------------------
Time: 2021-01-09 13:46:24
-------------------------------------------
191

-------------------------------------------
Time: 2021-01-09 13:46:29
-------------------------------------------
190

-------------------------------------------
Time: 2021-01-09 13:46:34
-------------------------------------------
200

-------------------------------------------
Time: 2021-01-09 13:46:39
-------------------------------------------
206

-------------------------------------------
Time: 2021-01-09 13:46:44
-------------------------------------------
203

-------------------------------------------
Time

Primero vemos que las salidas están espaciadas con 5 segundos, como era de esperar al haber indicado que el desplazamiento sea de 5 segundos.

El número de tweets se estabiliza alrededor de 200. Tiene sentido porque el productor al enviar un tweet espera un tiempo aleatorio entre 0 y 0.1s, por tanto, de media espera 0.05 segundos. Lo que quiere decir que en 10 segundo envía una media de 200 tweets (10/0.05 = 200). Por tanto el resultado que obtenemos se corresponde con el esperado.